# Example: Fitting Broad components and Narrow-Line Region Emission Lines 

This notebook steps through using the BEAT routine to fit multiple components to [O III] 4959,5007 and H-beta emission lines in PG_1307+085 which also includes some broad components. Spectra for this notebook are included in the 'spectra_pg' folder.

In [1]:
import beat
import numpy as np
import os
import matplotlib.pyplot as plt

# Reading in data

Here we define a function (load_file) to read a spectrum .txt file and create arrays for wavelength, flux, and noise (i.e. error). In this example, the .txt files being read only have 3 columns of data - wavelength, flux, and noise.

We use numpy.loadtxt to read in the data, but one can use any method that returns the required wavelength, flux, and noise arrays. 

In [2]:
def load_file(filepath):
    """reads file and returns wave, flux, and noise arrays to pass to fitting code"""
    wave, flux, noise = np.loadtxt(filepath, usecols=(0, 1, 2), unpack=True)
    return wave, flux, noise

# Take a look at your data
In this example, PG_1307+085 has an obvious broad feature that we would like to fit first before moving on to the NLR emission lines

In [ ]:
from IPython.display import display, Image
display(Image(filename='pg_example_plot.png'))

# Set definitions for broad emission lines

Preparing to fit the spectrum, we must define three sets of parameters concerning A) the target (target_param), B) the spectrum continuum (cont_instructions), and C) the spectrum features to be fit (fit_instructions).

In [ ]:
np.random.seed(1) # randseed set for consistency in this example notebook
def main():
    target_param = {'name': 'pg',      # Name of target
                    'red': 0.15434,    # Redshift of target
                    'minwidth': 1.125, # Minimum Gaussian sigma value to be fit to features [wavelength units]
                    'maxwidth': 40,    # Maximum Gaussian sigma value to be fit to features [wavelength units]
                    'start': 90,       # Minimum array value of used spectrum [array units]
                    'end': 900,        # Maximum array value of used spectrum [array units]
                    'fluxsigma': 3,    # The factor n multiplied by continuum standard deviation (i.e. n x std. dev.) to set minimum flux height. This is often 3.
                    'plotmin': 5420,   # The x-axis minimum wavelength in output plots [wavelength units]
                    'plotmax': 5960,   # The x-axis maximum wavelength in output plots [wavelength units]
                    'maxcomp': 5,      # The maximum number of components that should be attempted per line. This is often 3.
                    'lnz': 5.0,        # Minimum acceptable logarithm of the ratio of Bayesian evidences. This should usually be 5.
                    'cores' : 1}       # Number of processors that are free to be assigned to multiprocessing pool
    
    cont_instructions = {'form': 'model',            # Type of input continuum; currently 'model' only
                         'cont_poly':1,              # Degree of polynomial fit to continuum
                         'continuum1': (5455, 5515), # First flux bin sampled to derive continuum, ideally to one side of your features(s) [wavelength units]
                         'continuum2': (5802, 5860)} # Second flux bin sampled to derive continuum, ideally to the other side of your features(s) [wavelength units]
    fit_instructions = {
                        'line1': 
                            {'name': 'broadcomp', # Name of feature being fit
                             'wave': 5625,        # Expected centroid position [wavelength units] of feature
                             'minwave': 5550,     # Minimum wavelength of possible centroid position of fit
                             'wave_range': 125.0, # Range of wavelengths that are possible centroid positions (centroid of any component can fall between 'minwave' and 'minwave+wave_range')
                             'flux_free': True}   # True: Flux is allowed to vary freely, False: Flux is locked in with another line
                        }
    
    fit = beat.Fit(out_dir='pg_fits/',                  # Existing directory where fitting outputs should be placed
                   spec_dir='spectra_pg/',              # Directory containing spectra files for this fit
                   load_file=load_file,                 # Function that will read your files (load_file in this example)
                   target_param=target_param,           # Target parameter dictionary
                   cont_instructions=cont_instructions, # Continuum fitting instructions dictionary
                   fit_instructions=fit_instructions,   # Fit instructions dictionary
                   prefit_instructions=None             # There are no prefit_instructions in this example
                   )
    fit.mp_handler()

# Fit the spectra

To start fitting, we just call the 'main' function.

In [ ]:
main()

# Analyze the output

Take a look at the fit output text file which will be under 'pg_fits/pg_out' in this example.

In [ ]:
import pandas as pd
data = pd.read_csv('pg_fits/pg_out/pg_broadcomp.txt', sep=",", header=0)
data.head()

In [ ]:
print(f'Number of components fit: {data["ncomps"][0]}')
print(f'Wave, Width, Flux of first component: {data["wave_1"][0]}, {data["width_1"][0]}, {data["flux_1_A"][0]}')
print(f'Wave, Width, Flux of second component: {data["wave_2"][0]}, {data["width_2"][0]}, {data["flux_2_A"][0]}')

In [ ]:
from IPython.display import IFrame, display
filepath = "pg_fits/pg_out/plots/PG_1307+085_combined_broadcomp_2_posterior.pdf"
IFrame(filepath, width=900, height=400)

Now that we have these broad components fit, we can try fitting the NLR emission lines with the broad components as our prefit_instructions (already known fits).

# Set definitions for narrow emission lines and known broad features
To fit the spectrum, we will again define the three sets of parameters concerning A) the target (target_param), B) the spectrum continuum (cont_instructions), and C) the spectrum features to be fit (fit_instructions). But now since we already know that we have some broad components in our fit and we know their charateristics, we can include them as parameters concerning D) already known parameters (prefit_instructions).

In [3]:
def main():
    target_param = {'name': 'pg', 
                    'red': 0.15434, 
                    'minwidth': 1.43, 
                    'maxwidth': 16.35,
                    'start': 90, 
                    'end': 1110, 
                    'fluxsigma': 3, 
                    'plotmin': 5425, 
                    'plotmax': 5960, 
                    'maxcomp': 3, 
                    'lnz': 5.0, 
                    'cores' : 1} 
    
    cont_instructions = {'form': 'model', 
                         'cont_poly':1, 
                         'continuum1': (5455, 5515), 
                         'continuum2': (5830, 5890)}
    
    prefit_instructions = {'flux': 'unknown',       # Leave as unknown if you want the flux to be able to scale with every new fit
                           'comp1':                         # First broad component
                                   {'name':'broad1',        # Name of broad component
                                    'cen': 5605.7304,       # Centroid of 1st component (taken from output file)
                                    'width': 29.6579,       # Width of 1st component (taken from output file)
                                    'flux_free': True},     # Leave as true if you would like the component flux to change
                           'comp2':                 # Second broad component
                                   {'name': 'broad2',       # Name of broad component
                                    'cen': 5665.5117,       # Centroid of 2nd component (taken from output file)
                                    'width': 35.6489,       # Width of 2nd component (taken from output file)
                                    'flux_free': False,     # Leave as false if you would like the component to be locked with the first component
                                    'flux_locked_with': 'comp1', # Needed if flux_free=False, component that it is locked with in flux
                                    'flux_ratio': 1.57}}    # Flux ratio to keep the two components locked in (Flux_1 / Flux_2 = 9.16e-14 / 5.83e-14 = ~1.57)
    
    fit_instructions = {
                        'line1': # Instructions for narrow line component OIII (5008)
                                {'name': 'oiii', 
                                 'wave': 5008.239, 
                                 'minwave': 5763, 
                                 'wave_range': 40.0, 
                                 'flux_free': True},     # Allow flux to vary freely
                        'line2': # Instructions for narrow line component OIII (4960)
                                {'name': 'oiii', 
                                  'wave': 4960.295, 
                                  'flux_free': False,     # Lock flux with another component
                                  'flux_locked_with': 'line1', # Lock flux w/ 'line1' (OIII 5008)
                                  'flux_ratio': 3},       # Flux ratio between line1/line2 = 3
                        'line3': # Instructions for narrow line component H-Beta (4861)
                                {'name': 'h-beta', 
                                 'wave': 4861.33, 
                                 'minwave': 5589, 
                                 'wave_range': 40.0, 
                                 'flux_free': True}       # Allow flux to vary freely
    }
    
    fit = beat.Fit(out_dir='pg_fits2/',
                   spec_dir='spectra_pg/',
                   load_file=load_file,
                   target_param=target_param,
                   cont_instructions=cont_instructions,
                   fit_instructions=fit_instructions,
                   prefit_instructions=prefit_instructions
                   )
    fit.mp_handler()

In [4]:
main()

remaining to fit: 1
Fitting PG_1307+085_combined
Min Y: 1.11e-15
Max Y: 5.93e-15
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =    1
 *****************************************************
 ln(ev)=  -31092.626776652654      +/-   3.5509484008332025E-005
 Total Likelihood Evaluations:          200
 Sampling finished. Exiting MultiNest
PG_1307+085_combined: trying 1 component(s)
 *****************************************************
 MultiNest v3.10
 Copyright Farhan Feroz & Mike Hobson
 Release Jul 2015

 no. of live points =  200
 dimensionality =   10
 *****************************************************
 ln(ev)=  -151.73684605453585      +/-  0.29953654091261667     
 Total Likelihood Evaluations:        19460
 Sampling finished. Exiting MultiNest
PG_1307+085_combined: trying 2 component(s)
 *****************************************************
 MultiNest v3.


 MultiNest Warning!
 Parameter           14  of mode            1  is converging towards the edge of the prior.
 Parameter           18  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter           14  of mode            1  is converging towards the edge of the prior.
 Parameter           18  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter           10  of mode            1  is converging towards the edge of the prior.
 Parameter           14  of mode            1  is converging towards the edge of the prior.
 Parameter           18  of mode            1  is converging towards the edge of the prior.

 MultiNest Warning!
 Parameter           10  of mode            1  is converging towards the edge of the prior.
 Parameter           14  of mode            1  is converging towards the edge of the prior.
 Parameter           18  of mode            1  is converging towards the edge of the pri

# Analyze the output

Take a look at the fit output text file which will be under 'pg_fits2/pg_out' in this example.

In [ ]:
import pandas as pd
data = pd.read_csv('pg_fits2/pg_out/pg_oiii.txt', sep=",", header=0)
data.head()

In [ ]:
from IPython.display import IFrame, display
filepath = "pg_fits2/pg_out/plots/PG_1307+085_combined_oiii_2_posterior.pdf"
IFrame(filepath, width=900, height=400)

# Final Notes

Notes on the parameters in this example

<span style='color:Blue'> target_param :  </span>
   - <span style='color:Green'>name and redshift</span> -- rather straightforward, provide a name and redshift for the target

   - <span style='color:Green'>widths</span> -- these are the minimum and maximum sigma values allowed when generating Gaussians to fit the profile
       - <span style='color:Blue'>minwidth :</span> 
           - This value should be set as the minimum resolvable width of an emission line in your spectrum.  
           - The example data comes from a spectrograph with a Resolving Power of ~1889 at 5063.6 Angstroms (5006.8 * z+1).
           - delta_lambda = lambda_observed / (R * 2.355) - note: delta_lambda gives a FWHM and we need a sigma value, so we divide by 2.355.
           - 5063.6 Angstroms / (1889 * 2.355) = 1.125 Angstroms
       - <span style='color:Blue'>maxwidth :</span>
           - This value should be set to the maximum width expected in the spectra. Emission-line gas in the example spectra likely does not exceed FWHM = 2000 km/s, so we will use that to define the maximum width.
           - delta_lambda = lambda_observed * (FWHM / 2.355) / speed of light 
           - 5063.6 Angstroms * (2000 km/s / 2.355) / 2.998e5 km/s = 14.344 Angstroms
   - <span style='color:Green'>fluxsigma</span> -- This defines the minimum Gaussian height allowed when fitting a component. Using a value of 3 requires the minimum height of a component to be 3 times the standard deviation of the continuum regions you select in cont_instructions further below.
    
   - <span style='color:Green'>plotmin/max</span> -- the min/max wavelength values plotted in output files. 4900 - 5150 Angstroms encapsulates the three emission lines we want to fit and the adjacent continuua used to derive the fit continuum.
   
   - <span style='color:Green'>maxcomp</span> -- It is unlikely that there are more than 2 components in our example spectra, we will fit a maximum of 3 components to confirm the presence of 2 and check if 3 is more likely. A reminder: BEAT will stop fitting when a more complex model is less likely than a simpler model (e.g. when using 2 components is a less likely fit than 1 component). Thus, even if we set a maxcomp value of 6, the routine will likely only fit up to 3 components per spectrum.
   
   - <span style='color:Green'>cores</span> -- In this example, we only assign tasks to two computer processors to show the viability of multiprocessing on most computers. Currently, personal laptops usually have 4 processors, so try changing cores to 3 and noting the difference in time it takes to finish fitting the spectra. Always allow one free processor to continue using your computer!

   - <span style='color:Green'>lnz</span> -- The logarithm of the ratio of Bayesian evidences provides a guide to what constitutes a significant difference between models. Fischer et al. 2017, ApJ, 834, 30 gives an indepth discription of evidence values. |lnz| > 5 is used as strong evidence that a more complex model is superior, per Feroz, F., Hobson, M. P., & Bridges, M. 2009, MNRAS, 398, 1601.

<span style='color:Blue'> cont_instructions:  </span>
    
   - <span style='color:Green'>form</span> -- Future iterations of BEAT will allow the user to input specific continuua from files, currently generating a model is the only option.

   - <span style='color:Green'>cont_poly</span> -- The continuum is fit in this spectrum by selecting line-free regions of continuum and fitting an nth polynomial curve to the data. For this distribution, a simple linear fit is sufficient, so n = 1 here.
   
   - <span style='color:Green'>continuum1/2</span> -- These are the bins of line-free continuua in the spectrum. Ideally, one would select a continuum region on either side of the lines being fit to properly account for any continuum slope.
   
<span style='color:Blue'>fit_instructions:  </span> each of the three emission lines needs to be defined in the fit
   - <span style='color:Green'>line1</span> - This should always be our 'main' line; the brightest feature we are fitting, which in this case is [O III] 5007
        - <span style='color:blue'>name</span> -- Name for the feature, which will be used to name the output file, 'oiii' here
        - <span style='color:blue'>wave</span> -- Rest wavelength of the [O III] 5007 line.
        - <span style='color:blue'>flux_free</span> -- This is a logic switch on whether the flux of this line is reliant on another line. We are trying to measure the flux of this line, so we want it to be free to vary as needed. Therefore flux_free = True. 
        - <span style='color:blue'>minwave</span> -- Minimum centroid wavelength allowed when fitting Gaussians. We don't expect any [O III] 5007 components to have a centroid bluer than 5050 Angstroms, so we use that value here.
        - <span style='color:blue'>wave_range</span>  -- The size of the range, starting at the defined minwidth value, of permitted centroid positions. minwave + wave_range = the maximum centroid wavelength allowed when fitting Gaussians. We don't expect any [O III] 5007 components to have a centroid redder than 5075 Angstroms, so we use a wave_range value of 25 Angstroms.
        
   - <span style='color:Green'>line2</span> - This line will be assigned to [O III] 4959. Note, we do not need to fit this line, as it's part of a doublet with [O III] 5007. However, fitting it in tandem with [O III] 5007 may produce a better fit as componet parameters would need to agree between the two lines. 
        - <span style='color:blue'>name</span> -- Name for the feature, again 'oiii'
        - <span style='color:blue'>wave</span> -- Rest wavelength of the [O III] 4959 line. Difference in wavelength between this line and 'line1', [O III] 5007, is used in tandem with the redshift to calculate the centroid and widths at this second position.
        - <span style='color:blue'>flux_free</span> -- We will lock this flux to a fraction of another line, so flux_free = False. As [O III] 4959 is a doublet with [O III] 5007, we do not need to let the flux of this line be an open parameter.  
        - <span style='color:blue'>flux_locked_with</span> - We specify that the flux of this line is locked with 'line1', [O III] 5007.
        - <span style='color:blue'>flux_ratio</span> - This value specifies the difference in flux between the doublet, where flux_ratio = line1/line2 = 3 for this doublet.
        - <span style='color:blue'>minwave/wave_range</span> -- we do not define these parameters because this line is in step with 'line1'
        
   - <span style='color:Green'>line3</span> - This line will be assigned to H-beta 4862. We will assume that the H-beta emitting gas is originating from the same point as the [O III] gas, and therefore that the component fit for this line has the same width and systemic offset as the [O III] line. 
        - <span style='color:blue'>name</span> -- Name for the feature, 'hbeta'
        - <span style='color:blue'>wave</span> -- Rest wavelength of the H-beta line. Difference in wavelength between this line and 'line1', [O III] 5007, is used in tandem with the redshift to calculate the centroid and widths at this third position.
        - <span style='color:blue'>flux_free</span> -- We are trying to measure the flux of this line, so we want it to be free to vary as needed. Therefore flux_free = True.

<span style='color:Blue'>Call the 'Fit' class instance: </span> 
   - <span style='color:Green'>spec_dir</span> -- Define an input directory where the input spectra .txt files are located.
   - <span style='color:Green'>out_dir</span> -- Define an output directory where the output data, plots, and text files will be saved.
   - The remainder of the instance passes our target parameters and continuum and fit instructions.

<span style='color:Blue'>fit.mp_handler()</span>fit.mp_handler() -- Calls mp_handler to assign tasks to Multiple Processors and start the fits